# Data anonymizer


**Objetivo:** Debido a que los datos serán eventualmente compartidos con servicios externos, tales como Tableau, PowerBI u otros. Los datos deben ser anonimizados para prevenir filtraciones de identidad e información sensible

In [1]:
import pandas as pd


In [3]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('../data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('../data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


In [5]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('../data/20210909_Aisen_notificaciones_0600.csv', 
                   sep='~', # CSV con separador ''~''
                   low_memory=False
                  )

In [6]:
# Realiza merging de los archivos de interes

df_merged = df.merge(df_establecimiento, how='left', on='establecimiento_salud').merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )


In [7]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']


data_aysen_clean = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']==True)&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

In [8]:
# Transforma Fechas a formato DateTime 

data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion'], errors = 'coerce') 

data_aysen_clean['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_clean['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_clean['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_clean['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_clean['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_clean['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_3'], errors = 'coerce')

In [9]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)

<ipython-input-9-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-9-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [17]:
# Datos serán semianonimizados porque aún posee informacion para trazabilidad

data_aysen_clean_21_anonymous = data_aysen_clean_21.drop([
    'run_profesional',
    'nombre_profesional',
    'telefono_contacto_profesional',
    'email_contacto_profesional',
    'tipo_identificacion',
    'identificacion_paciente',
    'dv',
    'nombres_paciente',
    'primer_apellido_paciente',
    'segundo_apellido_paciente',
    'fecha_nacimiento',#existe columna edad
    'via_residencia',
    'direccion',
    'numero_residencia',
    'dpto_residencia',
    'poblacion_villa',
    'telefono_fijo',
    'telefono_celular',
    'descripcion_trabajo',
    'rubro_trabajo',
    'actividad_laboral_declarada',
    'nombre_lugar_trabajo_estudio',
    'direccion_lugar_trabajo_estudio'
    
], axis=1) 

In [18]:
data_aysen_clean_21_anonymous.to_csv('../data/df_notificaciones_anonimo.csv')